# Libraries

In [1]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [2]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\euro_goals.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,referee_nationality,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,Group 1,20:30,France,Denmark,1–0,Parc des Princes,Paris,47570,Volker Roth,West Germany,...,78,0,1,0,0,0,0,0,1984-06-12,12 June 1984
1,Group 1,20:30,Belgium,Yugoslavia,2–0,Stade Félix-Bollaert,Lens,41525,Erik Fredriksson,Sweden,...,28,0,2,0,0,0,0,0,1984-06-13,13 June 1984
2,Group 1,20:30,Belgium,Yugoslavia,2–0,Stade Félix-Bollaert,Lens,41525,Erik Fredriksson,Sweden,...,45,0,2,0,0,0,0,0,1984-06-13,13 June 1984
3,Group 1,17:15,France,Belgium,5–0,Stade de la Beaujoire,Nantes,51359,Bob Valentine,Scotland,...,4,0,5,0,0,0,0,0,1984-06-16,16 June 1984
4,Group 1,17:15,France,Belgium,5–0,Stade de la Beaujoire,Nantes,51359,Bob Valentine,Scotland,...,74,0,5,0,0,1,0,0,1984-06-16,16 June 1984


# Clean and transfrom variables

## time

In [3]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## date

In [4]:
# Convert 'short_date' column to datetime if it's not already
df['short_date'] = pd.to_datetime(df['short_date'])

# Extract the year from 'short_date' and create a new column 'year'
df['year'] = df['short_date'].dt.year

# Extract relevant columns

In [5]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
0,1984,Group 1,France,Denmark,France,78,1984-06-12,20:30,1–0
1,1984,Group 1,Belgium,Yugoslavia,Belgium,28,1984-06-13,20:30,2–0
2,1984,Group 1,Belgium,Yugoslavia,Belgium,45,1984-06-13,20:30,2–0
3,1984,Group 1,France,Belgium,France,4,1984-06-16,17:15,5–0
4,1984,Group 1,France,Belgium,France,74,1984-06-16,17:15,5–0


# Recreate Leauge Table after first two matchdays

### Filter out the matches from the last match day in the goals_df DataFrame.

In [6]:
# Step 1: Find the last match date for each tournament and group
last_dates = goals_df.groupby(['year', 'stage'])['short_date'].max().reset_index()

# Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
# Performing a left merge and keeping only rows not present in the last_dates
goals_before_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='left', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Create the dataset with only the last match day games (goals_last_day)
goals_last_day = goals_df.merge(last_dates, on=['year', 'stage', 'short_date'], how='inner')

# Step 4: Sort the DataFrame by 'goal_minute'
goals_last_day_sorted = goals_last_day.sort_values(by=['goal_minute'], ascending=True)

# Ensure that goals_last_day_sorted has no duplicates
goals_last_day_sorted = goals_last_day_sorted.drop_duplicates()

In [7]:
goals_last_day_sorted

,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
193,2021,Group E,Sweden,Poland,Sweden,2,2021-06-23,18:00,3–2
87,2004,Group A,Russia,Greece,Russia,2,2004-06-20,19:45,2–1
225,2024,Group F,Georgia,Portugal,Georgia,2,2024-06-26,21:00,2–0
17,1988,Group 2,England,Soviet Union,Soviet Union,3,1988-06-18,15:30,1–3
53,1996,Group D,Croatia,Portugal,Portugal,4,1996-06-19,16:30,0–3
...,...,...,...,...,...,...,...,...,...
195,2021,Group E,Sweden,Poland,Sweden,94,2021-06-23,18:00,3–2
213,2024,Group C,Slovenia,Serbia,Serbia,95,2024-06-20,15:00,1–1
76,2000,Group C,FR Yugoslavia,Spain,Spain,95,2000-06-21,18:00,3–4
211,2024,Group B,Croatia,Italy,Italy,98,2024-06-24,21:00,1–1


## create df  of aggregate data for each match

In [8]:
# Initialize an empty list to store the results
results = []

# Iterate over each match in the dataset
for match_id, group in goals_before_last_day.groupby(['year', 'stage', 'home_team', 'away_team']):
    # Initialize goals_home and goals_away for each match
    goals_home = 0
    goals_away = 0
    
    # Extract the local_time and the score from the first row in the group
    local_time = group.iloc[0]['local_time']
    score = group.iloc[0]['score']  # Assuming score is stored with a longer dash like '2–0'
    
    # Extract short_date from local_time (assuming local_time is a datetime object or string convertible to date)
    short_date = pd.to_datetime(local_time).date()  # Convert local_time to a short date (YYYY-MM-DD)
    
    # Loop through each row in the group to count goals for home and away teams
    for _, row in group.iterrows():
        # Check if the scorer's nationality matches the home team
        if row['home_team'] == row['scorer_nationality']:
            goals_home += 1
        # If the scorer's nationality matches the away team, increment goals_away
        elif row['away_team'] == row['scorer_nationality']:
            goals_away += 1

    # Calculate the expected score from goals_home and goals_away (using a standard hyphen)
    calculated_score = f"{goals_home}-{goals_away}"
    
    # Normalize both scores by replacing any long dash (en-dash or em-dash) with a standard hyphen
    normalized_score = score.replace("–", "-").replace("—", "-")
    
    # Check if the normalized score matches the calculated score
    score_match = normalized_score == calculated_score
    
    # Append the results to the list, including local_time, short_date, and score match flag
    results.append({
        'year': match_id[0],
        'stage': match_id[1],
        'home_team': match_id[2],
        'away_team': match_id[3],
        'local_time': local_time,
        'short_date': short_date,  # New column for the short date
        'goals_home': goals_home,
        'goals_away': goals_away,
        'original_score': score,  # Keep the original score with long dash
        'calculated_score': calculated_score,
        'score_match': score_match  # True if score matches, False otherwise
    })

# Convert the list into a DataFrame
agg_goals_before_last_day = pd.DataFrame(results)

# Add a new column 'won' based on the comparison of goals_home and goals_away
agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
    lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
    axis=1
)

In [9]:
agg_goals_before_last_day

,year,stage,home_team,away_team,local_time,short_date,goals_home,goals_away,original_score,calculated_score,score_match,won
0,1984,Group 1,Belgium,Yugoslavia,20:30,2024-10-18,2,0,2–0,2-0,True,1
1,1984,Group 1,Denmark,Yugoslavia,20:30,2024-10-18,5,0,5–0,5-0,True,1
2,1984,Group 1,France,Belgium,17:15,2024-10-18,5,0,5–0,5-0,True,1
3,1984,Group 1,France,Denmark,20:30,2024-10-18,1,0,1–0,1-0,True,1
4,1984,Group 2,Portugal,Spain,20:30,2024-10-18,1,1,1–1,1-1,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...
153,2024,Group E,Slovakia,Ukraine,15:00,2024-10-18,1,2,1–2,1-2,True,-1
154,2024,Group F,Georgia,Czech Republic,15:00,2024-10-18,1,1,1–1,1-1,True,0
155,2024,Group F,Portugal,Czech Republic,21:00,2024-10-18,2,1,2–1,2-1,True,1
156,2024,Group F,Turkey,Georgia,18:00,2024-10-18,3,1,3–1,3-1,True,1


## aggregate data for home and away games

In [10]:
# Step 1: Home games aggregation with match count
home_games = agg_goals_before_last_day.groupby(['year', 'stage', 'home_team']).agg(
    goals_scored=('goals_home', 'sum'),
    goals_conceded=('goals_away', 'sum'),
    points_home=('won', lambda x: sum(3 if result == 1 else (1 if result == 0 else 0) for result in x)),
    match_count_home=('home_team', 'count')  # Count how many times each team played at home
).reset_index()


In [11]:
display(home_games.head())

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home
0,1984,Group 1,Belgium,2,0,3,1
1,1984,Group 1,Denmark,5,0,3,1
2,1984,Group 1,France,6,0,6,2
3,1984,Group 2,Portugal,1,1,1,1
4,1984,Group 2,Romania,1,1,1,1


In [12]:
# Step 2: Away games aggregation with match count
away_games = agg_goals_before_last_day.groupby(['year', 'stage', 'away_team']).agg(
    goals_scored=('goals_away', 'sum'),
    goals_conceded=('goals_home', 'sum'),
    points_away=('won', lambda x: sum(3 if result == -1 else (1 if result == 0 else 0) for result in x)),
    match_count_away=('away_team', 'count')  # Count how many times each team played away
).reset_index()


In [13]:
display(away_games.head())

,year,stage,away_team,goals_scored,goals_conceded,points_away,match_count_away
0,1984,Group 1,Belgium,0,5,0,1
1,1984,Group 1,Denmark,0,1,0,1
2,1984,Group 1,Yugoslavia,0,7,0,2
3,1984,Group 2,Romania,1,2,0,1
4,1984,Group 2,Spain,2,2,2,2


## aggregate data after first two matches following 

### FIFA criteria (first aggregate level, then h2h) 

In [14]:

def fifa_before_last(home_games, away_games, agg_goals_before_last_day):
    # Step 1: Merge home_games and away_games on year, stage, and home_team with away_team
    all_games_before_last = pd.merge(
        home_games,
        away_games,
        left_on=['year', 'stage', 'home_team'],
        right_on=['year', 'stage', 'away_team'],
        how='outer',
        suffixes=('_home', '_away')
    )

    # Step 2: For teams that only appear in home_games, assign home_team to team and copy relevant columns
    all_games_before_last['team'] = all_games_before_last['home_team'].fillna(all_games_before_last['away_team'])

    # Step 3: For goals_scored, goals_conceded, and points, handle missing values
    all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
    all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
    all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)

    # Step 4: Add goals_difference column
    all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

    # Step 5: Sum match_count_home and match_count_away to get total match count for each team
    all_games_before_last['total_matches'] = all_games_before_last['match_count_home'].fillna(0) + all_games_before_last['match_count_away'].fillna(0)

    # Step 6: Adjust points if total_matches is 1 (indicating a likely 0-0 draw)
    all_games_before_last.loc[all_games_before_last['total_matches'] == 1, 'points'] += 1

    # Step 7: Drop unnecessary columns used in the merge process
    all_games_before_last = all_games_before_last[['year', 'stage', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']]

    # Step 8: Initial sort by year, stage, points, goals_difference, and goals_scored within each year and stage
    all_games_before_last = all_games_before_last.sort_values(
        by=['year', 'stage', 'points', 'goals_difference', 'goals_scored'],
        ascending=[True, True, False, False, False]
    ).reset_index(drop=True)

    # Step 9: Initialize a new column for tie-break results, which will store either the tie-break result or 'no need'
    all_games_before_last['tiebreaker'] = 'no need'

    # Step 10: Define the function to check the tie-breaker
    def check_tiebreaker(row1, row2, agg_data):
        """
        Check the tie-breaker based on the head-to-head match result from agg_goals_before_last_day.
        Return the team that won (if any) or 'tie'.
        """
        # Look for the head-to-head match between the two teams in agg_goals_before_last_day
        match = agg_data[((agg_data['home_team'] == row1['team']) & (agg_data['away_team'] == row2['team'])) |
                         ((agg_data['home_team'] == row2['team']) & (agg_data['away_team'] == row1['team']))]

        if not match.empty:
            if match.iloc[0]['won'] == 1:
                return row1['team']  # Home team won
            elif match.iloc[0]['won'] == -1:
                return row2['team']  # Away team won
            else:
                return 'tie'  # It's a draw
        else:
            return 'no result'  # No match found

    # Step 11: Loop through the sorted dataframe and apply the tie-breaker if needed
    for i in range(len(all_games_before_last) - 1):
        row1 = all_games_before_last.iloc[i]
        row2 = all_games_before_last.iloc[i + 1]

        # Check if the two rows have identical values for points, goal difference, and goals scored
        if (row1['points'] == row2['points'] and
            row1['goals_difference'] == row2['goals_difference'] and
            row1['goals_scored'] == row2['goals_scored']):

            # Apply the tie-breaker by checking the head-to-head match result
            tiebreak_result = check_tiebreaker(row1, row2, agg_goals_before_last_day)

            # Store the tie-breaker result for both teams involved
            all_games_before_last.at[i, 'tiebreaker'] = tiebreak_result
            all_games_before_last.at[i + 1, 'tiebreaker'] = tiebreak_result

    # Step 12: Add group standing by ranking teams within each stage based on points, goal difference, and goals scored
    all_games_before_last['standing'] = all_games_before_last.groupby(['year', 'stage']).apply(
        lambda x: x.sort_values(by=['points', 'goals_difference', 'goals_scored'], ascending=[False, False, False])
    ).reset_index(drop=True).groupby(['year', 'stage']).cumcount() + 1

    # Convert goals_scored, goals_conceded, points, goals_difference, and total_matches to integers
    all_games_before_last[['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']] = all_games_before_last[
        ['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']].astype(int)

    return all_games_before_last


In [15]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = fifa_before_last(home_games, away_games, agg_goals_before_last_day)


In [16]:
# Filter specific group
filtered_games = all_games_before_last[(all_games_before_last['stage'] == 'Group F') & 
                                       (all_games_before_last['year'] == 2021)]

# Display the filtered dataset
display(filtered_games)

,year,stage,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,standing
147,2021,Group F,France,2,1,4,1,2,no need,1
148,2021,Group F,Portugal,5,4,3,1,2,no need,2
149,2021,Group F,Germany,4,3,3,1,2,no need,3
150,2021,Group F,Hungary,1,4,1,-3,2,no need,4


# Recreate league table after last match day

### fifa criteria

In [17]:
def fifa_final(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    # Step 1: Filter the data for the specific year and stage
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['year'] == year) & 
        (all_games_before_last['stage'] == stage)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['year'] == year) & 
        (goals_last_day_sorted['stage'] == stage)
    ]

    # Step 2: Initialize columns for tracking team performance
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']

    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']
    group_goals_tracking['last_game_standing'] = 0
    group_goals_tracking['1st'] = 0
    group_goals_tracking['2nd'] = 0
    group_goals_tracking['3rd'] = 0
    group_goals_tracking['4th'] = 0
    group_goals_tracking['changes'] = 0  # Initialize this but will be redefined later as the sum of 1st, 2nd, 3rd, 4th
    group_goals_tracking['tied'] = False  # Initialize a flag to track tied teams
    group_goals_tracking['tied_won'] = 0  # Initialize to track if the team won a tie-breaker match

    # Step 3: Sort group_goals_last_day by goal_minute in ascending order
    group_goals_last_day = group_goals_last_day.sort_values(by='goal_minute', ascending=True)

    # Step 4: Iterate through the sorted and filtered last match goals and update the goals_tracking table
    previous_standings = group_goals_tracking['last_game_standing'].copy()
    first_iteration = True  # Variable to track the first iteration

    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home_team']
        away_team = goal['away_team']
        player_team = goal['scorer_nationality']

        # Print goal information for each goal
        print(f"Analyzing goal: {goal['goal_minute']} minute, Player team: {player_team}, Home: {home_team}, Away: {away_team}")

        # Update the goals based on who scored the goal (no own goal condition anymore)
        if player_team == home_team:
            # Home team scored, update home scored and away conceded
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
        elif player_team == away_team:
            # Away team scored, update away scored and home conceded
            group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
            group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1

        # Step 5: Update total_goals_scored, total_goals_conceded, and total_goal_difference
        group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored'] + group_goals_tracking['last_game_goals_scored']
        group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded'] + group_goals_tracking['last_game_goals_conceded']
        group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Step 6: Assign points for the last game dynamically after each goal
        for i, row in group_goals_tracking.iterrows():
            if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 3  # Win
            elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 1  # Draw
            else:
                group_goals_tracking.loc[i, 'last_game_points'] = 0  # Loss

        # Step 7: Update total points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']

        # Step 8: Mark teams that are tied
        group_goals_tracking['tied'] = group_goals_tracking.duplicated(subset=['total_points', 'total_goal_difference', 'total_goals_scored'], keep=False)

        # Reset `tied_won` to 0 for all teams
        group_goals_tracking['tied_won'] = 0

        # Step 8b: Calculate tied_won only for tied teams
        tied_teams = group_goals_tracking[group_goals_tracking['tied']]

        if not tied_teams.empty:
            # Iterate over tied teams to resolve standings using head-to-head results
            for index, row in tied_teams.iterrows():
                team1 = row['team']
                team1_index = index

                # Look for other teams tied with this team
                for other_index, other_row in tied_teams[tied_teams.index != index].iterrows():
                    team2 = other_row['team']
                    team2_index = other_index

                    # Check if these two teams played against each other in agg_goals_before_last_day
                    match = agg_goals_before_last_day[
                        ((agg_goals_before_last_day['home_team'] == team1) & (agg_goals_before_last_day['away_team'] == team2)) |
                        ((agg_goals_before_last_day['home_team'] == team2) & (agg_goals_before_last_day['away_team'] == team1))
                    ]

                    if not match.empty:
                        match_result = match.iloc[0]['won']  # Assume the 'won' column holds 1 for home win, -1 for away win, 0 for draw

                        # Resolve the tie using the match result
                        if match_result == 1:
                            # Home team won
                            if match.iloc[0]['home_team'] == team1:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 1
                            else:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 1
                        elif match_result == -1:
                            # Away team won
                            if match.iloc[0]['away_team'] == team1:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 1
                            else:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 1
                        elif match_result == 0:
                            # Draw, both teams get the same standing
                            group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 0
                            group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 0

        # Step 8c: Sort teams by total points, goal difference, goals scored, and tied_won
        group_goals_tracking = group_goals_tracking.sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored', 'tied_won'],
                                                                ascending=[False, False, False, False])

        # Step 9: Assign standings based on the sorting and tie resolution
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1

        # Step 10: Track changes and update standing positions after each goal is processed
        for i, row in group_goals_tracking.iterrows():
            team = row['team']

            if first_iteration:
                # Skip the update if the standing did not change
                if row['before_last_game_standing'] == row['last_game_standing']:
                    continue  # Skip updating the counters
                else:
                    # Update the position counters since standing has changed
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

                # Disable first iteration flag after the first goal
                first_iteration = False
            else:
                # Track the standing position counters only if the position differs from the previous state
                if row['last_game_standing'] != previous_standings[i]:  
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

        # Update previous standings after each goal
        previous_standings = group_goals_tracking['last_game_standing'].copy()

        # Step 11: Calculate changes as the sum of 1st, 2nd, 3rd, and 4th
        group_goals_tracking['changes'] = group_goals_tracking[['1st', '2nd', '3rd', '4th']].sum(axis=1)

        # Step 12: Print the updated group_goals_tracking after processing each goal
        print("\n=== Updated Standings After This Goal ===\n")
        display_columns = ['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 
                           'total_goal_difference', 'last_game_points', 'last_game_standing', 
                           'changes', '1st', '2nd', '3rd', '4th', 'tied', 'tied_won']
        print(group_goals_tracking[display_columns].to_string(index=False))
        print("\n========================================\n")

    # Step 13: Return the final DataFrame
    return group_goals_tracking


In [18]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year and stage
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    result = fifa_final(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[['year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]


Analyzing goal: 26 minute, Player team: Belgium, Home: Denmark, Away: Belgium

=== Updated Standings After This Goal ===

      team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_points  last_game_standing  changes  1st  2nd  3rd  4th  tied  tied_won
    France             7                   6                     0                      6                 1                   1        0    0    0    0    0 False         0
   Belgium             6                   3                     5                     -2                 3                   2        1    0    1    0    0 False         0
   Denmark             3                   5                     2                      3                 0                   3        1    0    0    1    0 False         0
Yugoslavia             1                   0                     7                     -7                 1                   4        1    0    0    0    1 False         0


Analyzing g

In [19]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\tb_euro_fifa.xlsx'
changes_df_euro.to_excel(file_path, index=False)